In [ ]:
import logging
import os
import sys
from typing import NoReturn

from arguments import DataTrainingArguments, ModelArguments
from datasets import DatasetDict, load_from_disk, load_metric, load_dataset,concatenate_datasets
from trainer_qa import QuestionAnsweringTrainer
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)
from utils_qa import check_no_error, postprocess_qa_predictions

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

In [ ]:
datasets = load_from_disk('/opt/ml/input/data/train_dataset')

In [ ]:
column_names = datasets["train"].column_names
question_column_name = "question" 
context_column_name = "context"
answer_column_name = "answers"
pad_on_right = tokenizer.padding_side == "right"

In [61]:
def prepare_train_features(examples):
    # truncation과 padding(length가 짧을때만)을 통해 toknization을 진행하며, stride를 이용하여 overflow를 유지합니다.
    # 각 example들은 이전의 context와 조금씩 겹치게됩니다.
    tokenized_examples = tokenizer(
        examples[context_column_name if pad_on_right else question_column_name],
        truncation=True,
        max_length=256,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        #return_token_type_ids=False, # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
        padding=True,
    )
    tokenized_questions = tokenizer(
        examples[question_column_name if pad_on_right else context_column_name],
        max_length=512,
        pad_to_max_length=True,
        padding="max_length",
    )
    
    # 길이가 긴 context가 등장할 경우 truncate를 진행해야하므로, 해당 데이터셋을 찾을 수 있도록 mapping 가능한 값이 필요합니다.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # token의 캐릭터 단위 position를 찾을 수 있도록 offset mapping을 사용합니다.
    # start_positions과 end_positions을 찾는데 도움을 줄 수 있습니다.
    offset_mapping = tokenized_examples.pop("offset_mapping")
    # 데이터셋에 "start position", "enc position" label을 부여합니다.
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    tokenized_examples["q_input_ids"] =[]
    tokenized_examples["q_token_type_ids"] =[]
    tokenized_examples["q_attention_mask"] = []
    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)  # cls index
        # sequence id를 설정합니다 (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        # 하나의 example이 여러개의 span을 가질 수 있습니다.
        sample_index = sample_mapping[i]
        answers = examples[answer_column_name][sample_index]
        tokenized_examples["q_input_ids"].append(tokenized_questions['input_ids'][sample_index])
        tokenized_examples["q_token_type_ids"].append(tokenized_questions['token_type_ids'][sample_index])
        tokenized_examples["q_attention_mask"].append(tokenized_questions['attention_mask'][sample_index])
        # answer가 없을 경우 cls_index를 answer로 설정합니다(== example에서 정답이 없는 경우 존재할 수 있음).
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # text에서 정답의 Start/end character index
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            answer = answers['text'][0]
            tokenized_answer = tokenizer.encode(answer)[1:-1]
            # text에서 current span의 Start token index
            token_start_index = 0
            # text에서 current span의 End token index
            token_end_index = len(input_ids) - 1

            # 정답이 span을 벗어났는지 확인합니다(정답이 없는 경우 CLS index로 label되어있음).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # token_start_index 및 token_end_index를 answer의 끝으로 이동합니다.
                # Note: answer가 마지막 단어인 경우 last offset을 따라갈 수 있습니다(edge case).
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)
    return tokenized_examples


In [62]:
train_dataset = datasets['train']
train_dataset = train_dataset.map(
            prepare_train_features,
            batched=True,
            remove_columns=column_names,
        )

In [63]:
train_dataset

Dataset({
    features: ['attention_mask', 'end_positions', 'input_ids', 'q_attention_mask', 'q_input_ids', 'q_token_type_ids', 'start_positions', 'token_type_ids'],
    num_rows: 13492
})

In [64]:
len(train_dataset['q_token_type_ids'][0])

512

In [39]:
import torch
a=torch.randn([32,512,768])
b=torch.randn([32,512,768])
c=a*b

In [40]:
c.size()

torch.Size([32, 512, 768])

In [42]:
a=torch.tensor([[[1,2,3],[4,5,6],[7,8,9],[0,1,2]],
                [[1,2,3],[4,5,6],[7,8,9],[0,1,2]]])

b=torch.tensor([[[0,1,0],[1,1,1],[2,2,2],[-1,1,-1]],
                [[1,2,3],[4,5,6],[7,8,9],[0,1,2]]])

In [43]:
a*b

tensor([[[ 0,  2,  0],
         [ 4,  5,  6],
         [14, 16, 18],
         [ 0,  1, -2]],

        [[ 1,  4,  9],
         [16, 25, 36],
         [49, 64, 81],
         [ 0,  1,  4]]])